In [ ]:
pip install lxml

In [ ]:
pip install html5lib

In [ ]:
pip install BeautifulSoup4

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_html('http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', match="Borough")[0]
df.columns = ["PostalCode", "Borough", "Neighborhood"]
df['Borough'].replace("Not assigned", np.nan, inplace=True)
df.dropna(subset=['Borough'], inplace=True)
df.shape

In [ ]:
df['Neighborhood'] = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].transform(lambda x: ','.join(x))
df['Neighborhood'].replace("Not assigned", df['Borough'], inplace=True)
df.drop_duplicates(inplace=True)
df.shape

In [ ]:
geo = pd.read_csv('http://cocl.us/Geospatial_data')
geo.rename(columns={"Postal Code" : "PostalCode"}, inplace=True)
df_geo = pd.merge(df, geo, how="inner", on="PostalCode")
df_geo

In [10]:
df_geo.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [9]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.6.14
  latest version: 4.7.10

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /srv/conda/envs/notebook

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    asn1crypto-0.24.0          |        py37_1003         154 KB  conda-forge
    certifi-2019.6.16          |           py37_1         149 KB  conda-forge
    cffi-1.12.3                |   py37h8022711_0         218 KB  conda-forge
    chardet-3.0.4              |        py37_1003         167 KB  conda-forge
    cryptography-2.7           |   py37h72c5cf5_0         610 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    idna-2.8                   |        py37_1000         100 

In [ ]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

In [15]:

address = 'Toronto City'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_geo['Latitude'], df_geo['Longitude'], df_geo['Borough'], df_geo['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto